In [1]:
import pandas as pd
import numpy as np
import chess
from stockfish import Stockfish
from timeit import default_timer as timer
import IPython.display as display
import os


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.utils import shuffle

import encode_data
import positional_features

device = 'cuda' if torch.cuda.is_available() else 'cpu'
stockfish = Stockfish(path = 'C:\Program Files\stockfish\stockfish-windows-x86-64-avx2.exe', depth=13, parameters={'Threads': 4, 'Hash': 32})

positions = torch.from_numpy(np.asarray(np.load('../prepared_data/positions.npy'))).to(device)
moves = torch.from_numpy(np.asarray(np.load('../prepared_data/moves.npy'))).to(device)

TRAIN_INDEX = int(len(positions) * 0.7)
BATCH_SIZE = 16

train_data = TensorDataset(positions[:TRAIN_INDEX], moves[:TRAIN_INDEX])
test_data = TensorDataset(positions[TRAIN_INDEX:], moves[TRAIN_INDEX:])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [2]:
class ChessLoss(nn.Module):
    def __init__(self):
        super(ChessLoss, self).__init__()
    
    def forward(self, predictions, labels):
        return

In [3]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.c1 = nn.Conv2d(256, 256, 3, 1, 1)
        self.b1 = nn.BatchNorm2d(256)
        self.c2 = nn.Conv2d(256, 256, 3, 1, 1)
        self.b2 = nn.BatchNorm2d(256)
        
    def forward(self, x):
        res = x
        out = F.relu(self.b1(self.c1(x)))
        out = self.b2(self.c2(out))
        out += res
        out = F.relu(out)
        return out
        

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.c1 = nn.Conv2d(14, 256, 3, 1, 1)
        self.b1 = nn.BatchNorm2d(256)
        
        for i in range(10):
            setattr(self, f'res{i}', ResNet())
        
        self.f1 = nn.Linear(256 * 8 * 8, 1024)
        self.f2 = nn.Linear(1024, 512)
        self.f3 = nn.Linear(512, 4672)
        
    def forward(self, x):
        x = x.to(torch.float32)
        x = x.permute(0, 3, 1, 2)
        
        x = F.relu(self.b1(self.c1(x)))
        
        for i in range(10):
            x = getattr(self, f'res{i}')(x)
        
        x = torch.flatten(x, 1)
        
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        x = self.f3(x)
        
        return x

Net()

Net(
  (c1): Conv2d(14, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (b1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res0): ResNet(
    (c1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (b1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (c2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (b2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res1): ResNet(
    (c1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (b1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (c2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (b2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res2): ResNet(
    (c1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (b1): Batch

In [10]:
net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.003)

NUM_EPOCHS = 20
len_loader = len(train_loader)
curr_loss = 0.0

start = timer()
for epoch in range(NUM_EPOCHS):
    total_loss = 0.0

    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        if i % 25 == 0:
            display.clear_output()
            print(f'[{i}/{len_loader}] epoch {epoch}/{NUM_EPOCHS} loss: {curr_loss:.4f} ')
        
        # if i == len(train_loader) - 1:
    display.clear_output()
    curr_loss = total_loss / len_loader
    print(f'epoch {epoch + 1}/{NUM_EPOCHS} loss: {curr_loss:.4f}')
    total_loss = 0.0

end = timer()
print(f'finished training in {(end - start):.2f}s')

epoch 20/20 loss: 0.1569
finished training in 4410.56s


In [11]:
PATH = '../models/resnet10.pth'
torch.save(net.state_dict(), PATH)

In [12]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
    
        outputs = net(inputs)    
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'model accuracy: {correct / total:.3f}')
# terrible accuracy currently due to lack of legal move check (wip)

model accuracy: 0.203


In [22]:
fen = 'rnbqk2r/pp3ppp/3bpn2/2pp4/3P1P2/3BP3/PPPN2PP/R1BQK1NR w KQkq - 0 6'
board_tensor = torch.from_numpy(np.asarray(encode_data.board_to_tensor(fen))).to(device)
board_tensor = board_tensor.unsqueeze(0)

net.eval()
out = net(board_tensor)
out = out.squeeze(0)

_, idx = torch.topk(out, 3)

moves = [encode_data.decodeMove(x, chess.Board(fen)) for x in idx.tolist()]
moves 

[Move.from_uci('c2c3'), Move.from_uci('g1f3'), Move.from_uci('d1e2')]